In [3]:
from bs4 import BeautifulSoup
import requests

# This is Graph Search Problem 
# I'll use Depth First Search

def is_valid_link(link, context=0):
    return ( (link != None) and (len(link) > 6) and (link[0] != "#") and ( link[:6] == "/wiki/" ) )


starting_page_title = ".wiki/Dolph_Lundgren"
page_title = starting_page_title
visited = {}

while ( page_title.lower() != "/wiki/philosophy" ):
    page_url = "https://en.wikipedia.org" + page_title
    html_page = requests.get( page_url ).content
    soup = BeautifulSoup( html_page, 'html.parser')
    out_links = [ link.get('href') for link in soup.find_all('a') if \
                 ( is_valid_link( link.get('href'), context=0 ) ) and\
                 ( link.get('href') not in visited ) \
                ] 
    if (len( out_links ) == 0):
        break
    else:
        page_title = out_links[0]
        visited[ page_title ] = True
        
        if ( len(visited) % 50 == 0 ):
            print( "Number of pages visited: {}".format( len(visited) ) )

if ("philosophy" in str.lower(page_title) ):
    print("Traversed from page '{}' to Philosophy after taking {} links.".format( starting_page_tile, len(visited) + 1 ) )
else:
    print("We reached a dead end page, congratulations. You proved the hypothesis false!")

# game = PathFinderGame()
# game.start()

Number of pages visited: 50
Number of pages visited: 100
Number of pages visited: 150
Number of pages visited: 200
Number of pages visited: 250
Number of pages visited: 300
Number of pages visited: 350
Number of pages visited: 400
Number of pages visited: 450
Number of pages visited: 500
Number of pages visited: 550
Number of pages visited: 600
Number of pages visited: 650
Number of pages visited: 700
Number of pages visited: 750
Number of pages visited: 800
We reached a dead end page, congratulations. You proved the hypothesis false!


In [9]:
starting_page_title = "/wiki/Dolph_Lundgren"
page_title = starting_page_title
page_url = "https://en.wikipedia.org" + page_title
html_page = requests.get( page_url ).content
soup = BeautifulSoup( html_page, 'html.parser')
soup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Dolph Lundgren - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgMonthNamesShort":["","Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"],"wgRequestId":"XfMVAApAIDEAAKNc@tMAAACG","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"Dolph_Lundgren","wgTitle":"Dolph Lundgren","wgCurRevisionId":930278241,"wgRevisionId":930278241,"wgArticleId":599745,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","CS1 Portuguese-language sources (pt)","Pages 

In [2]:
class PathFinderGame:
    def __init__(self):
        self.starting_page_title = None
        return None
    
    '''
    !!
    '''
    def start(self):
        while ( self.starting_link == None ):
            proposed_starting_page_title = input("Enter the title of the wiki page you'd like to start from: ")
            
            if ( self.verify_wiki_page_title( proposed_starting_link ) == True ):
                self.starting_page_title = proposed_starting_link
            else:
                print("There is no wiki page that exists with that title")
        
        # dfs stuff
        
        return None
    
        
    '''
        bool : 
        
        Given a page title as a String,
        Checks whether a Wikipedia Page with that title exists  
    '''
    def verify_wiki_page_title(self, title):
        return True
        

True